<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/test_finQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CSE8803_DLT_Project/FinQA/

/content/drive/MyDrive/CSE8803_DLT_Project/FinQA


In [3]:
from PyPDF2 import PdfReader
import pandas as pd
import io

def extract_text_and_table(pdf_path):
    # Initialize a PDF reader
    reader = PdfReader(pdf_path)

    # Variables to store text before and after table
    pre_text, post_text = "", ""
    table_started = False
    table_ended = False
    table_lines = []

    # Iterate through each page
    for page in reader.pages:
        # Extract text from the page
        text = page.extract_text()

        if text:
            lines = text.split('\n')
            for line in lines:
                # Identify the start and end of the table
                if 'Table' in line:
                    table_started = True
                    continue
                if table_started and line.strip() == '':
                    table_ended = True

                # Collecting text before, inside, and after the table
                if not table_started:
                    pre_text += line + '\n'
                elif table_started and not table_ended:
                    table_lines.append(line)
                elif table_ended:
                    post_text += line + '\n'

    # Converting table lines to DataFrame for better visualization
    table_data = io.StringIO('\n'.join(table_lines))
    df = pd.read_fwf(table_data)

    return pre_text, post_text

In [4]:
%ls

code/  dataset/  eg-intro.png  LICENSE  README.md  test_finQA.ipynb  test_image.pdf  test_qa.pdf


In [25]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 32.9 MB/s eta 0:00:00


In [5]:
from tabula import read_pdf
import pandas as pd

def extract_tables_from_pdf_tabula(pdf_path):
    # Using tabula to read tables from the PDF
    tables = read_pdf(pdf_path, pages='all', multiple_tables=True, lattice=True, stream=True)

    # Convert tables to a readable text format
    table_texts = []
    for table in tables:
        if isinstance(table, pd.DataFrame):
            table_texts.append(table.to_string(index=False))

    return '\n\n'.join(table_texts)

# # Path to the PDF file
# pdf_path = '/mnt/data/test_image.pdf'

# # Extract table text using tabula
# extracted_table_text = extract_tables_from_pdf_tabula(pdf_path)
# extracted_table_text


In [26]:
print(extract_tables_from_pdf_tabula('./test_qa_2.pdf'))

                   ( dollars in millions )              2006                      2005              2004
net cash provided by operating\ractivities          $ 1410.5                  $ 1143.3          $ 1229.0
                   additions to properties -453.1 ( 453.1\r)          -374.2 ( 374.2 ) -278.6 (\r278.6 )
                                 cash flow           $ 957.4                   $ 769.1           $ 950.4
                      year-over-yearchange 24.5% ( 24.5\r% ) 221219.1% ( 221219.1\r% )               NaN

Empty DataFrame
Columns: [Prev]
Index: []


In [10]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 36.1 MB/s eta 0:00:00


In [9]:
!pip install fitz
!pip install frontend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [11]:
import fitz  # PyMuPDF
from tabula import read_pdf
import pandas as pd

def extract_contents_from_pdf(pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)

    pre_text, post_text = "", ""
    table_started = False

    # Use tabula to read tables from the PDF
    tables = read_pdf(pdf_path, pages='all', multiple_tables=True, lattice=True, stream=True, silent=True)
    table_texts = [table.to_string(index=False) for table in tables if isinstance(table, pd.DataFrame)]

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text().split('\n')

        for line in text:
            if any(table_text in line for table_text in table_texts):
                table_started = True
                continue

            if not table_started:
                pre_text += line + '\n'
            else:
                post_text += line + '\n'

    return pre_text, '\n\n'.join(table_texts), post_text


In [20]:
pre_text, table_texts, post_text = extract_contents_from_pdf('./test_qa_2.pdf')

In [23]:
print(f'''Pre Text''')
print(f'{pre_text}')
print('-'.join('' for i in range(100)))
print(f'''Post Text''')
print(f'{post_text}')
print('-'.join('' for i in range(100)))
print(f'''Table Text''')
print(f'{table_texts}')
print('-'.join('' for i in range(100)))

Pre Text
11/16/23, 4:05 PM
Explore FinQA
https://ﬁnqasite.github.io/explore.html
1/2
Explore FinQA
abiomed , inc . and subsidiaries notes to consolidated ﬁnancial statements
2014 ( continued ) note 14 . income taxes ( continued ) and transition and
deﬁnes the criteria that must be met for the beneﬁts of a tax position to be
recognized . as a result of its adoption of ﬁn no . 48 , the company has
recorded the cumulative eﬀect of the change in accounting principle of $ 0.3
million as a decrease to opening retained earnings and an increase to other
long-term liabilities as of april 1 , 2007 . this adjustment relates to state nexus
for failure to ﬁle tax returns in various states for the years ended march 31 ,
2003 , 2004 , and 2005 . the company has initiated a voluntary disclosure
plan . the company has elected to recognize interest and/or penalties related
to income tax matters in income tax expense in its consolidated statements
of operations . as of april 1 , 2007 , accrued interest w

In [19]:
extract_tables_from_pdf_tabula('./test_qa.pdf')

'                                                          balance at april 1 2007      $ 224\nreductions for tax positions for closing of the applicable statute of limitations -56 ( 56 )\n                                                         balance at march 31 2008      $ 168'